In [7]:
from qiskit import *
from qiskit.circuit import ParameterVector

In [8]:
from src.xyz_evolution import XYZEvolutionCircuit
qc = XYZEvolutionCircuit(4)
_ = qc.uxyz(1, 2, 3, 1, 2)
qc.draw()

q_0: ─────────
     ┌───────┐
q_1: ┤0      ├
     │  Uxyz │
q_2: ┤1      ├
     └───────┘
q_3: ─────────

In [9]:
A = ParameterVector('t',3)

In [10]:
def qc_(theta1,theta2,theta3):
    qc = QuantumCircuit(1)
    qc.rx(theta1,0)
    qc.ry(theta2,0)
    qc.rz(theta3,0)
    return qc

In [11]:
qc_(*A)

In [12]:
import numpy as np

from math import ceil
from qiskit import QuantumCircuit
from qiskit.circuit import ParameterVector
from src.propagators import UXZGate, UXYZGate
from src.xyz_evolution import XYZEvolutionCircuit

In [13]:
import numpy as np

# Assuming 'measurements' is a dictionary
measurements = {'temperature': 25, 'humidity': 60, 'pressure': 1013}

# Define the dtype for the resulting NumPy array
dtype = [('key', 'U10'), ('value', int)]  # Data type for the array elements (string for keys, int for values)

# Convert dictionary items to tuples and create a NumPy array
result_array = np.fromiter(
    map(lambda kv: (kv[0], kv[1]), measurements.items()),  # Map each (key, value) pair to a tuple
    dtype=dtype  # Set the data type for the resulting array
)

print(result_array)

[('temperatur',   25) ('humidity',   60) ('pressure', 1013)]


In [14]:
 measurements.items()

dict_items([('temperature', 25), ('humidity', 60), ('pressure', 1013)])

In [15]:
for i in map(lambda kv: (kv[0], kv[1]), measurements.items()):
    print(i)

('temperature', 25)
('humidity', 60)
('pressure', 1013)


In [16]:
import numpy as np

# 0부터 9까지의 정수를 가진 리스트를 생성
my_list = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

# 리스트로부터 NumPy 배열 생성
result_array = np.fromiter(my_list, dtype=int)

print(result_array)  # 생성된 NumPy 배열 출력

[0 1 2 3 4 5 6 7 8 9]


In [17]:
import numpy as np
from qiskit.quantum_info import SparsePauliOp


class StaggeredMagnetization:
    r"""
    Calculator for the time-dependent staggered magnetization of a spin
    chain evolving under the action of a Heisenberg Hamiltonian.

    In particular, this class implements a ``__call__`` dunder method that
    evaluates the expectation value defining $M_{\mathrm{stag}}(t)$ with
    respect to the quantum state described by the ``measurements``
    dictionary of ``(state, counts)`` key-value pairs.

    EXAMPLES::

        >>> from qiskit_aer import Aer
        >>> from src.staggered_magnetization import StaggeredMagnetization
        >>> from src.xyz_evolution import XYZEvolutionCircuit
        >>> num_qubits, t = 3, 1
        >>> qc = XYZEvolutionCircuit(num_qubits, [1, 1, 1], final_time=t, trotter_num=10)
        >>> backend = Aer.get_backend("aer_simulator")
        >>> qc = qc.decompose(["Uxz", "Uxyz"])
        >>> qc.measure_all()
        >>> psi = backend.run(qc).result().get_counts()
        >>> m_stag = StaggeredMagnetization(num_qubits)
        >>> m_stag(psi)
        0.3333333333333333
    """
    def __init__(self, num_qubits):
        terms = [("Z", [j], (-1)**j) for j in range(num_qubits)]
        self.hamiltonian = 1 / num_qubits * SparsePauliOp.from_sparse_list(terms, num_qubits)

    def __call__(self, measurements):
        r"""
        Evaluate the time-dependent staggered magnetization $m_s(t)$ defined by
        :ref:`Equation (1) <stag_mag>` with respect to the quantum state
        $\vert \psi(t) \rangle$ described by the dictionary ``measurements``
        of ``(state, count)`` key-value pairs.
        """
        n = self.hamiltonian.num_qubits
        dtype = np.dtype([("states", int, (n,)), ("counts", "f")])
        res = np.fromiter(map(lambda kv: (list(kv[0]), kv[1]), measurements.items()), dtype)
        shots = res["counts"].sum()
        return np.dot(self.energies(res["states"]), res["counts"]) / shots

    def energies(self, states):
        """
        Quickly obtain eigenvalues of an Ising Hamiltonian corresponding to the
        given ``states``.

        EXAMPLES::

            >>> import numpy as np
            >>> from src.staggered_magnetization import StaggeredMagnetization
            >>> m_stag = StaggeredMagnetization(5)
            >>> states = np.array([[0, 1, 1, 0, 1], [1, 1, 0, 0, 0]])
            >>> m_stag.energies(states)
            array([-0.2,  0.2])
        """
        paulis = np.array([list(str(ops)) for ops in self.hamiltonian.paulis]) != "I"
        coeffs = self.hamiltonian.coeffs.real
        energies = [0]*len(states)
        temp = np.matmul(states,paulis)
        temp = np.where(temp==1,-1,temp)
        temp = np.where(temp==0,1,temp)
        energies = np.sum(coeffs*temp,axis=1)
        return energies

In [18]:
from qiskit_aer import Aer
from src.staggered_magnetization import StaggeredMagnetization
from src.xyz_evolution import XYZEvolutionCircuit
num_qubits, t = 3, 1
qc = XYZEvolutionCircuit(num_qubits, [1, 1, 1], final_time=t, trotter_num=10)
backend = Aer.get_backend("aer_simulator")
qc = qc.decompose(["Uxz", "Uxyz"])
qc.measure_all()
psi = backend.run(qc).result().get_counts()
m_stag = StaggeredMagnetization(num_qubits)
m_stag(psi)

0.3333333333333333

In [19]:
import numpy as np
from src.staggered_magnetization import StaggeredMagnetization
m_stag = StaggeredMagnetization(5)
states = np.array([[0, 1, 1, 0, 1], [1, 1, 0, 0, 0]])
m_stag.energies(states)

array([-0.2,  0.2])

In [20]:
from src.xyz_evolution import XYZEvolutionCircuit
from qiskit.circuit import ParameterVector

temp = ParameterVector('t',6)
XYZEvolutionCircuit(4, [1, 1, 1], trotter_num=2).draw()

┌───────┐         ┌───────┐         
q_0: ┤0      ├─────────┤0      ├─────────
     │  Uxyz │┌───────┐│  Uxyz │┌───────┐
q_1: ┤1      ├┤0      ├┤1      ├┤0      ├
     ├───────┤│  Uxyz │├───────┤│  Uxyz │
q_2: ┤0      ├┤1      ├┤0      ├┤1      ├
     │  Uxyz │└───────┘│  Uxyz │└───────┘
q_3: ┤1      ├─────────┤1      ├─────────
     └───────┘         └───────┘

In [21]:
equiv_qc = XYZEvolutionCircuit(3)
equiv_qc.uxz(*temp[0:2], 1, 2)
equiv_qc.draw()

q_0: ───────────────────
     ┌─────────────────┐
q_1: ┤0                ├
     │  Uxz(t[0],t[1]) │
q_2: ┤1                ├
     └─────────────────┘

In [22]:
target_qc = XYZEvolutionCircuit(4)

for index,params in enumerate(np.array([[0,1],[0,2],[0,1],[0,2],[0,5],[0,1]])):
    index = index%(target_qc.num_qubits-1)
    if index<int(target_qc.num_qubits/2-1):
        target_qc.uxz(*params, 2*index+1, 2*index + 2)
    else:
        target_qc.uxz(*params, 2*index-2, 2*index - 1)

target_qc.draw()

┌──────┐        ┌──────┐
q_0: ────────┤0     ├────────┤0     ├
     ┌──────┐│  Uxz │┌──────┐│  Uxz │
q_1: ┤0     ├┤1     ├┤0     ├┤1     ├
     │  Uxz │├──────┤│  Uxz │├──────┤
q_2: ┤1     ├┤0     ├┤1     ├┤0     ├
     └──────┘│  Uxz │└──────┘│  Uxz │
q_3: ────────┤1     ├────────┤1     ├
             └──────┘        └──────┘

In [23]:
for i in np.array([[1,2],[2,3],[3,4]]):
    print(*i)

1 2
2 3
3 4


In [24]:
np.array([[1,2],[2,3],[3,4]])

array([[1, 2],
       [2, 3],
       [3, 4]])

In [25]:
import numpy as np
import random as rnd

from copy import deepcopy
from qiskit.quantum_info import Operator
from scipy.optimize import minimize
from src.xyz_evolution import XYZEvolutionCircuit
from qiskit.circuit import ParameterVector

import numpy as np
import random as rnd

from copy import deepcopy
from qiskit.quantum_info import Operator
from scipy.optimize import minimize
from src.xyz_evolution import XYZEvolutionCircuit
from qiskit.circuit import ParameterVector

import numpy as np
import random as rnd

from copy import deepcopy
from qiskit.quantum_info import Operator
from scipy.optimize import minimize
from src.xyz_evolution import XYZEvolutionCircuit
from qiskit.circuit import ParameterVector

In [26]:
import numpy as np
import random as rnd

from copy import deepcopy
from qiskit.quantum_info import Operator
from scipy.optimize import minimize
from src.xyz_evolution import XYZEvolutionCircuit
from qiskit.circuit import ParameterVector

import numpy as np
import random as rnd

from copy import deepcopy
from qiskit.quantum_info import Operator
from scipy.optimize import minimize
from src.xyz_evolution import XYZEvolutionCircuit
from qiskit.circuit import ParameterVector

class CircuitCompressor:
    """
    A class that implements the YBE-powered QTD circuit compression scheme.
    """
    def __init__(self, xyz_evolution_qc):
        self.deep_qc = xyz_evolution_qc

    def get_ybe_update(self, params, l2r=True):
        r"""
        Update $3$-qubit time propagator circuit parameters according to the
        Yang-Baxter Equation (YBE).


        INPUT:

            - ``params`` -- a (3, 2) array describing the parameters for each
              of the three blocks
            - ``l2r`` -- a boolean indicating whether we are applying the symmetry
              left-to-right or vice versa

        OUTPUT:

            A (3, 2) NumPy array describing the parameters of an equivalent circuit. 
        """
        if l2r:
            # Get equivalent parametrized circuit
            equiv_qc = XYZEvolutionCircuit(3)
            params = np.reshape(params,[3,2])
            result_params = [ParameterVector('theta'+str(k),2) for k in range(3)]
            for index,param in enumerate(result_params):
                index = index%(equiv_qc.num_qubits-1)
                if index<int(equiv_qc.num_qubits/2):
                    equiv_qc.uxz(*param, 2*index, 2*index + 1)
                else:
                    index = index - int(equiv_qc.num_qubits/2)
                    equiv_qc.uxz(*param, 2*index+1, 2*index + 2)


            # Construct target unitary operator
            target_qc = XYZEvolutionCircuit(3)
            
            for index,param in enumerate(params):
                index = index%(target_qc.num_qubits-1)
                if index<int(target_qc.num_qubits/2-1):
                    target_qc.uxz(*param, 2*index+1, 2*index + 2)
                else:
                    index = index - int(equiv_qc.num_qubits/2)
                    target_qc.uxz(*param, 2*index, 2*index +1)

            target = Operator(target_qc).data

            # Get parameters for the equivalent time propagator
            new_params = get_optimized_params(equiv_qc, target)
            #new_params = np.reshape(new_params,[3,2])
            return new_params
        else:
            # Get equivalent parametrized circuit
            equiv_qc = XYZEvolutionCircuit(3)
            params = np.reshape(params,[3,2])
            result_params = [ParameterVector('theta'+str(k),2) for k in range(3)]
            for index,param in enumerate(result_params):
                index = index%(equiv_qc.num_qubits-1)
                if index<int(equiv_qc.num_qubits/2):
                    equiv_qc.uxz(*param, 2*index+1, 2*index + 2)
                else:
                    index = index - int(equiv_qc.num_qubits/2)
                    equiv_qc.uxz(*param, 2*index, 2*index + 1)


            # Construct target unitary operator
            target_qc = XYZEvolutionCircuit(3)
            
            for index,param in enumerate(params):
                index = index%(target_qc.num_qubits-1)
                if index<int(target_qc.num_qubits/2):
                    target_qc.uxz(*param, 2*index, 2*index + 1)
                else:
                    index = index - int(equiv_qc.num_qubits/2)
                    target_qc.uxz(*param, 2*index+1, 2*index +2)

            target = Operator(target_qc).data

            # Get parameters for the equivalent time propagator
            new_params = get_optimized_params(equiv_qc, target)
            #new_params = np.reshape(new_params,[3,2])
            return new_params
            

    def get_mirror_update(self, params, l2r=True):
        r"""
        Update the $4$-qubit time propagator circuit parameters following the
        so-called mirror step.

        INPUT:

            - ``params`` -- a (6, 2) NumPy array describing the parameters for
              each of the six blocks
            - ``l2r`` -- a boolean indicating whether we are applying the symmetry
              left-to-right or vice versa
        
        OUTPUT:

            A (6, 2) NumPy array describing the parameters of the mirrored circuit.
        """
        # Number of parameters per block
        bsz = 2
        params = np.reshape(params,[6,2])
        equiv_qc = XYZEvolutionCircuit(4)
        
        result_params = [ParameterVector('theta'+str(k),bsz) for k in range(6)]
        for index,param in enumerate(result_params):
            index = index%(equiv_qc.num_qubits-1)
            if index<int(equiv_qc.num_qubits/2):
                equiv_qc.uxz(*param, 2*index, 2*index + 1)
            else:
                index = index - int(equiv_qc.num_qubits/2)
                equiv_qc.uxz(*param, 2*index+1, 2*index + 2)


        # Construct target unitary operator
        target_qc = XYZEvolutionCircuit(4)
        
        for index,param in enumerate(params):
            index = index%( target_qc.num_qubits-1)
            if index<int(target_qc.num_qubits/2-1):
                target_qc.uxz(*param, 2*index+1, 2*index + 2)
            else:
                index = index - int(target_qc.num_qubits/2)
                target_qc.uxz(*param, 2*index, 2*index +1)

        target = Operator(target_qc).data

        # Get parameters for the equivalent time propagator
        new_params = get_optimized_params(equiv_qc, target)
        
        return new_params

    def compress_circuit(self):
        """
        Compress this time evolution circuit using the YBE.

        OUTPUT:

            Returns a compressed :class:`.XYZEvolutionCircuit` object
            with $N (N - 1)/2$ blocks that is equivalent to the uncompressed
            Trotterization circuit ``self.deep_qc``.
        """
        # Return if deep_qc is empty
        qc = self.deep_qc
        if not list(qc):
            return qc

        # Extract parameters
        bsz = 2
        evol_params = qc.time_delta * qc.coupling_const[np.nonzero(qc.coupling_const)]
        N = qc.num_qubits
        w = np.zeros(N*(N-1))

        #####################
        ### Fill this in! ###
        #####################

        # Construct compressed circuit
        compressed = XYZEvolutionCircuit(N)
        compressed._construct_evolution_qc(qc.propagator, num_layers=N, bound=False)
        return compressed.assign_parameters(w)

#########################
### Parameter fitting ###
#########################

# Minimize quadratic target loss... Attempt max_shot times using random initial point
def get_optimized_params(param_circ, target_unitary, res_tol=1e-7, max_shots=10, verbose=False):
    # Define quadratic loss for fitting
    loss = lambda p: np.linalg.norm(Operator(param_circ.assign_parameters(p)).data - target_unitary)
    n_params = param_circ.num_parameters
    min_loss = 100
    w_opt = 0
    for k in range(max_shots):
        w0 = [2*np.pi*rnd.random()+np.pi for j in range(n_params)]
        opt = minimize(
                loss, w0, method='L-BFGS-B', bounds=[(0, 4*np.pi)]*n_params,
                jac='3-point', options={'ftol': 1e-10}
            )
        if opt.fun < min_loss:
            min_loss = opt.fun
            w_opt = opt.x
        if min_loss < res_tol:
            return w_opt
    if verbose:
        print("WARNING: Good fitting parameters were not found! Residual loss is {}".format(min_loss))
    return w_opt



In [27]:
from qiskit import QuantumCircuit
import numpy as np
from math import ceil
from qiskit import QuantumCircuit
from qiskit.circuit import ParameterVector
from src.propagators import UXZGate, UXYZGate
class XYZEvolutionCircuit(QuantumCircuit):
    """
    A ``QuantumCircuit`` that implements time propagation under the action
    of a Heisenberg Hamiltonian.

    EXAMPLES::

        >>> from src.xyz_evolution import XYZEvolutionCircuit
        >>> XYZEvolutionCircuit(4, [1, 1, 1], trotter_num=2).draw()
             ┌───────┐         ┌───────┐         
        q_0: ┤0      ├─────────┤0      ├─────────
             │  Uxyz │┌───────┐│  Uxyz │┌───────┐
        q_1: ┤1      ├┤0      ├┤1      ├┤0      ├
             ├───────┤│  Uxyz │├───────┤│  Uxyz │
        q_2: ┤0      ├┤1      ├┤0      ├┤1      ├
             │  Uxyz │└───────┘│  Uxyz │└───────┘
        q_3: ┤1      ├─────────┤1      ├─────────
             └───────┘         └───────┘         
        >>> qc = XYZEvolutionCircuit(3, [1, 0, 1], magnetic_field=1, trotter_num=2, final_time=3)
        >>> qc.draw()
              ░ ┌─────────┐ ░ ┌──────┐         ░ ┌─────────┐ ░ ┌──────┐        
        q_0: ─░─┤ Rz(1.5) ├─░─┤0     ├─────────░─┤ Rz(1.5) ├─░─┤0     ├────────
              ░ ├─────────┤ ░ │  Uxz │┌──────┐ ░ ├─────────┤ ░ │  Uxz │┌──────┐
        q_1: ─░─┤ Rz(1.5) ├─░─┤1     ├┤0     ├─░─┤ Rz(1.5) ├─░─┤1     ├┤0     ├
              ░ ├─────────┤ ░ └──────┘│  Uxz │ ░ ├─────────┤ ░ └──────┘│  Uxz │
        q_2: ─░─┤ Rz(1.5) ├─░─────────┤1     ├─░─┤ Rz(1.5) ├─░─────────┤1     ├
              ░ └─────────┘ ░         └──────┘ ░ └─────────┘ ░         └──────┘
    """
    def __init__(
            self, 
            num_qubits,
            coupling_const=None,
            magnetic_field=0,
            final_time=0,
            trotter_num=0,
            bound=True,
        ):
        # Initialize QuantumCircuit
        super().__init__(num_qubits)

        # Record Hamiltonian parameters
        for param in ["coupling_const", "magnetic_field", "final_time", "trotter_num"]:
            setattr(self, param, eval(param))
        self.time_delta = final_time / trotter_num if trotter_num else 0
        self.magnetic = magnetic_field
        # Impute the appropriate propagator
        self.propagator = "".join("XYZ"[j] for j in np.nonzero(coupling_const)[0])
        
        # Construct evolution circuit
        self._construct_evolution_qc(bound=coupling_const is not None)

    def uxz(self, gamma, delta, qubit1, qubit2):
        r"""
        Apply the parametrized :class:`.UXZGate` onto ``qubit1`` and ``qubit2``.
        
        EXAMPLES::

            >>> from src.xyz_evolution import XYZEvolutionCircuit
            >>> qc = XYZEvolutionCircuit(3)
            >>> _ = qc.uxz(1, 2, 0, 1)
            >>> _ = qc.uxz(1, 2, 1, 2)
            >>> qc.draw()
                 ┌──────┐        
            q_0: ┤0     ├────────
                 │  Uxz │┌──────┐
            q_1: ┤1     ├┤0     ├
                 └──────┘│  Uxz │
            q_2: ────────┤1     ├
                         └──────┘
        """
        time_prop = UXZGate(gamma, delta).to_instruction()
        return self.append(time_prop, [qubit1, qubit2])

    def uxyz(self, thetax, thetay, thetaz, qubit1, qubit2):
        r"""
        Apply the parametrized :class:`.UXYZGate` onto ``qubit1`` and ``qubit2``.

        EXAMPLES::

            >>> from src.xyz_evolution import XYZEvolutionCircuit
            >>> qc = XYZEvolutionCircuit(4)
            >>> _ = qc.uxyz(1, 2, 3, 1, 2)
            >>> qc.draw()

            q_0: ─────────
                 ┌───────┐
            q_1: ┤0      ├
                 │  Uxyz │
            q_2: ┤1      ├
                 └───────┘
            q_3: ─────────
        """
        time_prop = UXYZGate(thetax, thetay, thetaz).to_instruction()
        return self.append(time_prop, [qubit1, qubit2])

    def _construct_evolution_qc(self, propagator=None, num_layers=None, odd=False, bound=True):
        r"""
        Construct the time propagator corresponding to evolution under the $XYZ$
        Hamiltonian.
        """
        # Set the number of layers if None is specified
        if num_layers is None:
            num_layers = 2 * self.trotter_num

        # Determine block operator and number of parameters per block
        if propagator is None:
            propagator = self.propagator
        block = getattr(self, "u" + propagator.lower())
        num_block_params = len(propagator)

        # Get parameters or bindings
        num_blocks = ceil(num_layers * (self.num_qubits - 1) / 2)
        if bound:
            J = np.array([Ja for Ja in self.coupling_const if not np.isclose(Ja, 0)])
            params = np.tile(self.time_delta * J, (num_blocks, 1))
        else:
            params = [ParameterVector("θ" + str(k), num_block_params) for k in range(num_blocks)]

        # Lay down blocks one step at a time!
        if not(odd):
            if self.magnetic == 0:
                for index,para in enumerate(params):
                    index = index%(self.num_qubits-1)
                    if index<int(self.num_qubits/2):
                        block(*para,2*index,2*index+1)
                    else:
                        index = index-int(self.num_qubits/2)
                        try:
                            block(*para,2*index+1,2*index+2)
                        except:
                            continue
            else:
                for index,para in enumerate(params):
                    index = index%(self.num_qubits-1)
                    if index == 0:
                        for i in range(self.num_qubits):
                            if i == 0:
                                self.barrier(self.qubits)
                            self.rz(self.time_delta*self.magnetic,i)
                            if i == self.num_qubits-1:
                                self.barrier(self.qubits)
                                
                    if index<int(self.num_qubits/2):
                        block(*para,2*index,2*index+1)
                    else:
                        index = index-int(self.num_qubits/2)
                        block(*para,2*index+1,2*index+2)
        else:
            if self.magnetic == 0:
                for index,para in enumerate(params):
                    index = index%(self.num_qubits-1)
                    if index<ceil(self.num_qubits/2-1):
                        block(*para,2*index+1,2*index+2)
                    else:
                        index = index-int(self.num_qubits/2)
                        try:
                            block(*para,2*index,2*index+1)
                        except:
                            continue
            else:
                for index,para in enumerate(params):
                    index = index%(self.num_qubits-1)
                    if index == 0:
                        for i in range(self.num_qubits):
                            if i == 0:
                                self.barrier(self.qubits)
                            self.rz(self.time_delta*self.magnetic,i)
                            if i == self.num_qubits-1:
                                self.barrier(self.qubits)
                                
                    if index<ceil(self.num_qubits/2-1):
                        block(*para,2*index+1,2*index+2)
                    else:
                        index = index-int(self.num_qubits/2)
                        block(*para,2*index,2*index+1)

In [15]:
from src.circuit_compressor import CircuitCompressor
params = np.random.rand(12)
temp = CircuitCompressor(XYZEvolutionCircuit(4)).get_mirror_update(params,False)

In [ ]:
temp

array([ 6.73039961,  3.22435141,  3.37463193,  6.31743478,  5.38926199,
       11.2983355 ,  1.45331885,  3.9941061 ,  4.32109401,  6.75380203,
        6.69643452,  6.52312199])

In [16]:
from src.xyz_evolution import XYZEvolutionCircuit
num_qubits = 4
rhs = XYZEvolutionCircuit(num_qubits)
rhs._construct_evolution_qc('XZ', num_layers=num_qubits,odd=True, bound=False)
rhs = rhs.assign_parameters(np.reshape(np.array(params),[-1]))
rhs.draw()
Operator(rhs)



os = XYZEvolutionCircuit(num_qubits)
os._construct_evolution_qc('XZ', num_layers=num_qubits, odd=False, bound=False)
os = os.assign_parameters(temp)
os.draw()
print(np.linalg.norm(Operator(os)- Operator(rhs)))
Operator(os) ==  Operator(rhs)

8.643341347240607e-06


False

In [ ]:
Operator(rhs) == Operator(os)

True

In [ ]:
rhs = XYZEvolutionCircuit(4)
rhs._construct_evolution_qc('XZ', num_layers=4,odd=True, bound=False)
rhs.draw()

┌───────────────────┐                     »
q_0: ─────────────────────┤0                  ├─────────────────────»
     ┌───────────────────┐│  Uxz(θ2[0],θ2[1]) │┌───────────────────┐»
q_1: ┤0                  ├┤1                  ├┤0                  ├»
     │  Uxz(θ0[0],θ0[1]) │├───────────────────┤│  Uxz(θ3[0],θ3[1]) │»
q_2: ┤1                  ├┤0                  ├┤1                  ├»
     └───────────────────┘│  Uxz(θ1[0],θ1[1]) │└───────────────────┘»
q_3: ─────────────────────┤1                  ├─────────────────────»
                          └───────────────────┘                     »
«     ┌───────────────────┐
«q_0: ┤0                  ├
«     │  Uxz(θ5[0],θ5[1]) │
«q_1: ┤1                  ├
«     ├───────────────────┤
«q_2: ┤0                  ├
«     │  Uxz(θ4[0],θ4[1]) │
«q_3: ┤1                  ├
«     └───────────────────┘

In [ ]:
from src.xyz_evolution import XYZEvolutionCircuit

os = XYZEvolutionCircuit(4)
os._construct_evolution_qc('XZ', num_layers=4, odd=False, bound=False)

os.draw()

┌───────────────────┐                     ┌───────────────────┐»
q_0: ┤0                  ├─────────────────────┤0                  ├»
     │  Uxz(θ0[0],θ0[1]) │┌───────────────────┐│  Uxz(θ3[0],θ3[1]) │»
q_1: ┤1                  ├┤0                  ├┤1                  ├»
     ├───────────────────┤│  Uxz(θ2[0],θ2[1]) │├───────────────────┤»
q_2: ┤0                  ├┤1                  ├┤0                  ├»
     │  Uxz(θ1[0],θ1[1]) │└───────────────────┘│  Uxz(θ4[0],θ4[1]) │»
q_3: ┤1                  ├─────────────────────┤1                  ├»
     └───────────────────┘                     └───────────────────┘»
«                          
«q_0: ─────────────────────
«     ┌───────────────────┐
«q_1: ┤0                  ├
«     │  Uxz(θ5[0],θ5[1]) │
«q_2: ┤1                  ├
«     └───────────────────┘
«q_3: ─────────────────────
«

In [ ]:
not(True)

False

In [ ]:
rhs = XYZEvolutionCircuit(num_qubits)
rhs._construct_evolution_qc('XZ', num_layers=num_qubits, bound=False)
rhs = rhs.assign_parameters(np.reshape(np.array([[1,2],[2,3],[4,5]]),[-1]))
rhs.draw()
Operator(rhs)

Operator([[ 0.0420468 +0.02496049j,  0.        +0.j        ,
            0.        +0.j        , -0.49541439+0.21000048j,
            0.        +0.j        , -0.837683  -0.02369166j,
            0.04336729+0.06259869j,  0.        +0.j        ],
          [ 0.        +0.j        ,  0.01530047+0.04644196j,
           -0.51146153-0.16716079j,  0.        +0.j        ,
            0.83264325-0.09475927j,  0.        +0.j        ,
            0.        +0.j        , -0.05176722-0.05585224j],
          [ 0.        +0.j        ,  0.49541439+0.21000048j,
            0.0420468 -0.02496049j,  0.        +0.j        ,
           -0.04336729+0.06259869j,  0.        +0.j        ,
            0.        +0.j        , -0.837683  +0.02369166j],
          [ 0.51146153-0.16716079j,  0.        +0.j        ,
            0.        +0.j        ,  0.01530047-0.04644196j,
            0.        +0.j        ,  0.05176722-0.05585224j,
            0.83264325+0.09475927j,  0.        +0.j        ],
          [ 0.      

In [ ]:
from qiskit.quantum_info import Operator, SparsePauliOp

In [ ]:
qc = XYZEvolutionCircuit(3)
qc.uxz(*[1,2],1,2)
qc.uxz(*[2,3],0,1)
qc.uxz(*[4,5],1,2)
Operator(qc)

Operator([[ 0.0420468 +0.02496049j,  0.        +0.j        ,
            0.        +0.j        ,  0.04336729+0.06259869j,
            0.        +0.j        , -0.837683  -0.02369166j,
           -0.49541439+0.21000048j,  0.        +0.j        ],
          [ 0.        +0.j        ,  0.01530047-0.04644196j,
            0.05176722-0.05585224j,  0.        +0.j        ,
            0.83264325+0.09475927j,  0.        +0.j        ,
            0.        +0.j        ,  0.51146153-0.16716079j],
          [ 0.        +0.j        , -0.04336729+0.06259869j,
            0.0420468 -0.02496049j,  0.        +0.j        ,
            0.49541439+0.21000048j,  0.        +0.j        ,
            0.        +0.j        , -0.837683  +0.02369166j],
          [-0.05176722-0.05585224j,  0.        +0.j        ,
            0.        +0.j        ,  0.01530047+0.04644196j,
            0.        +0.j        , -0.51146153-0.16716079j,
            0.83264325-0.09475927j,  0.        +0.j        ],
          [ 0.      

In [ ]:
from src.xyz_evolution import XYZEvolutionCircuit
qc = XYZEvolutionCircuit(3)
t = ParameterVector('t',6)
qc.uxz(t[0],t[1],0,1)
qc.uxz(t[2],t[3],1,2)
qc.uxz(t[4],t[5],0,1)
qc = qc.assign_parameters(np.reshape(np.array([[1,2],[2,3],[4,5]]),[-1]))
print(qc)
Operator(qc)

     ┌───────────┐             ┌───────────┐
q_0: ┤0          ├─────────────┤0          ├
     │  Uxz(1,2) │┌───────────┐│  Uxz(4,5) │
q_1: ┤1          ├┤0          ├┤1          ├
     └───────────┘│  Uxz(2,3) │└───────────┘
q_2: ─────────────┤1          ├─────────────
                  └───────────┘             


Operator([[ 0.0420468 +0.02496049j,  0.        +0.j        ,
            0.        +0.j        , -0.49541439+0.21000048j,
            0.        +0.j        , -0.837683  -0.02369166j,
            0.04336729+0.06259869j,  0.        +0.j        ],
          [ 0.        +0.j        ,  0.01530047+0.04644196j,
           -0.51146153-0.16716079j,  0.        +0.j        ,
            0.83264325-0.09475927j,  0.        +0.j        ,
            0.        +0.j        , -0.05176722-0.05585224j],
          [ 0.        +0.j        ,  0.49541439+0.21000048j,
            0.0420468 -0.02496049j,  0.        +0.j        ,
           -0.04336729+0.06259869j,  0.        +0.j        ,
            0.        +0.j        , -0.837683  +0.02369166j],
          [ 0.51146153-0.16716079j,  0.        +0.j        ,
            0.        +0.j        ,  0.01530047-0.04644196j,
            0.        +0.j        ,  0.05176722-0.05585224j,
            0.83264325+0.09475927j,  0.        +0.j        ],
          [ 0.      

In [ ]:
temp[0]

array([7.95895452, 3.77573325])

In [ ]:
import numpy as np
import random as rnd
from copy import deepcopy
from qiskit.quantum_info import Operator
from scipy.optimize import minimize
from src.xyz_evolution import XYZEvolutionCircuit
from qiskit.circuit import ParameterVector

import numpy as np
import random as rnd

from copy import deepcopy
from qiskit.quantum_info import Operator
from scipy.optimize import minimize
from src.xyz_evolution import XYZEvolutionCircuit
from qiskit.circuit import ParameterVector

class CircuitCompressor:
    """
    A class that implements the YBE-powered QTD circuit compression scheme.
    """
    def __init__(self, xyz_evolution_qc):
        self.deep_qc = xyz_evolution_qc

    def get_ybe_update(self, params, l2r=True):
        r"""
        Update $3$-qubit time propagator circuit parameters according to the
        Yang-Baxter Equation (YBE).


        INPUT:

            - ``params`` -- a (3, 2) array describing the parameters for each
              of the three blocks
            - ``l2r`` -- a boolean indicating whether we are applying the symmetry
              left-to-right or vice versa

        OUTPUT:

            A (3, 2) NumPy array describing the parameters of an equivalent circuit. 
        """
        if l2r:
            # Get equivalent parametrized circuit
            equiv_qc = XYZEvolutionCircuit(3)
            params = np.reshape(params,[3,2])
            result_params = [ParameterVector('theta'+str(k),2) for k in range(3)]
            for index,param in enumerate(result_params):
                index = index%(equiv_qc.num_qubits-1)
                if index<int(equiv_qc.num_qubits/2):
                    equiv_qc.uxz(*param, 2*index, 2*index + 1)
                else:
                    index = index - int(equiv_qc.num_qubits/2)
                    equiv_qc.uxz(*param, 2*index+1, 2*index + 2)


            # Construct target unitary operator
            target_qc = XYZEvolutionCircuit(3)
            
            for index,param in enumerate(params):
                index = index%(target_qc.num_qubits-1)
                if index<int(target_qc.num_qubits/2-1):
                    target_qc.uxz(*param, 2*index+1, 2*index + 2)
                else:
                    index = index - int(equiv_qc.num_qubits/2)
                    target_qc.uxz(*param, 2*index, 2*index +1)

            target = Operator(target_qc).data

            # Get parameters for the equivalent time propagator
            new_params = get_optimized_params(equiv_qc, target)
            #new_params = np.reshape(new_params,[3,2])
            return new_params
        else:
            # Get equivalent parametrized circuit
            equiv_qc = XYZEvolutionCircuit(3)
            params = np.reshape(params,[3,2])
            result_params = [ParameterVector('theta'+str(k),2) for k in range(3)]
            for index,param in enumerate(result_params):
                index = index%(equiv_qc.num_qubits-1)
                if index<int(equiv_qc.num_qubits/2):
                    equiv_qc.uxz(*param, 2*index+1, 2*index + 2)
                else:
                    index = index - int(equiv_qc.num_qubits/2)
                    equiv_qc.uxz(*param, 2*index, 2*index + 1)


            # Construct target unitary operator
            target_qc = XYZEvolutionCircuit(3)
            
            for index,param in enumerate(params):
                index = index%(target_qc.num_qubits-1)
                if index<int(target_qc.num_qubits/2):
                    target_qc.uxz(*param, 2*index, 2*index + 1)
                else:
                    index = index - int(equiv_qc.num_qubits/2)
                    target_qc.uxz(*param, 2*index+1, 2*index +2)

            target = Operator(target_qc).data

            # Get parameters for the equivalent time propagator
            new_params = get_optimized_params(equiv_qc, target)
            #new_params = np.reshape(new_params,[3,2])
            return new_params
            

    def get_mirror_update(self, params, l2r=True):
        r"""
        Update the $4$-qubit time propagator circuit parameters following the
        so-called mirror step.

        INPUT:

            - ``params`` -- a (6, 2) NumPy array describing the parameters for
            each of the six blocks
            - ``l2r`` -- a boolean indicating whether we are applying the symmetry
            left-to-right or vice versa
        
        OUTPUT:

            A (6, 2) NumPy array describing the parameters of the mirrored circuit.
        """
        # Number of parameters per block
        if l2r == False:
            equiv_qc = XYZEvolutionCircuit(4)
            equiv_qc._construct_evolution_qc('XZ', num_layers=4,odd=False, bound=False)
            # Construct target unitary operator
            target_qc = XYZEvolutionCircuit(4)
            target_qc._construct_evolution_qc('XZ', num_layers=4,odd=True, bound=False)
            target_qc = target_qc.assign_parameters(params)
            target = Operator(target_qc).data
        else:
            equiv_qc = XYZEvolutionCircuit(4)
            equiv_qc._construct_evolution_qc('XZ', num_layers=4,odd=True, bound=False)
            # Construct target unitary operator
            target_qc = XYZEvolutionCircuit(4)
            target_qc._construct_evolution_qc('XZ', num_layers=4,odd=False, bound=False)
            target_qc = target_qc.assign_parameters(params)
            target = Operator(target_qc).data
            

        # Get parameters for the equivalent time propagator
        new_params = get_optimized_params(equiv_qc, target)
        
        return new_params


    def compress_circuit(self):
        """
        Compress this time evolution circuit using the YBE.

        OUTPUT:

            Returns a compressed :class:`.XYZEvolutionCircuit` object
            with $N (N - 1)/2$ blocks that is equivalent to the uncompressed
            Trotterization circuit ``self.deep_qc``.
        """
        # Return if deep_qc is empty
        qc = self.deep_qc
        if not list(qc):
            return qc

        # Extract parameters
        bsz = 2
        evol_params = qc.time_delta * qc.coupling_const[np.nonzero(qc.coupling_const)]

        N = qc.num_qubits
        w = np.zeros(N*(N-1))
        if(N == 3):
            num_layers = 2 * self.deep_qc.trotter_num
            num_blocks = ceil(num_layers * (self.deep_qc.num_qubits - 1) / 2)
            params = list(evol_params)*num_blocks
            for i in range(num_blocks-int(N*(N-1)/2)):
                if i == 0:
                    update = self.get_ybe_update(params[0:int(N*(N-1))])
                    update[4] += evol_params[0]
                    update[5] += evol_params[1] 
                else:
                    update = self.get_ybe_update(update)
                    update[4] += evol_params[0]
                    update[5] += evol_params[1]
            if (num_blocks-3)%2 == 1:
                compressed = XYZEvolutionCircuit(N)
                compressed._construct_evolution_qc(qc.propagator, num_layers=N,odd=True, bound=False)
            else:
                compressed = XYZEvolutionCircuit(N)
                compressed._construct_evolution_qc(qc.propagator, num_layers=N, bound=False)    
            w = update
            return compressed.assign_parameters(w)
            
        if(N == 4):
            num_layers = 2 * self.deep_qc.trotter_num
            num_blocks = ceil(num_layers * (self.deep_qc.num_qubits - 1) / 2)
            params = list(evol_params)*num_blocks
            for i in range(num_blocks-int(N*(N-1)/2)):
                if i == 0:
                    update = self.get_mirror_update(params[0:int(N*(N-1))])
                    update[8] += evol_params[0]
                    update[9] += evol_params[1] 
                    update[10] += evol_params[0] 
                    update[11] += evol_params[1] 
                elif i%2 == 1:
                    update = self.get_mirror_update(update,False)
                    update[10] += evol_params[0]
                    update[11] += evol_params[1]
                else:
                    update = self.get_mirror_update(update)
                    update[8] += evol_params[0]
                    update[9] += evol_params[1] 
                    update[10] += evol_params[0] 
                    update[11] += evol_params[1] 
                    
            if (num_blocks-3)%2 == 1:
                compressed = XYZEvolutionCircuit(N)
                compressed._construct_evolution_qc(qc.propagator, num_layers=N,odd=True, bound=False)
            else:
                compressed = XYZEvolutionCircuit(N)
                compressed._construct_evolution_qc(qc.propagator, num_layers=N, bound=False)    
                
            w = update
        # Construct compressed circuit
            return compressed.assign_parameters(w)

#########################
### Parameter fitting ###
#########################

# Minimize quadratic target loss... Attempt max_shot times using random initial point
def get_optimized_params(param_circ, target_unitary, res_tol=1e-7, max_shots=10, verbose=False):
    # Define quadratic loss for fitting
    loss = lambda p: np.linalg.norm(Operator(param_circ.assign_parameters(p)).data - target_unitary)
    n_params = param_circ.num_parameters
    min_loss = 100
    w_opt = 0
    for k in range(max_shots):
        w0 = [2*np.pi*rnd.random()+np.pi for j in range(n_params)]
        opt = minimize(
                loss, w0, method='L-BFGS-B', bounds=[(0, 4*np.pi)]*n_params,
                jac='3-point', options={'ftol': 1e-10}
            )
        if opt.fun < min_loss:
            min_loss = opt.fun
            w_opt = opt.x
        if min_loss < res_tol:
            return w_opt
    if verbose:
        print("WARNING: Good fitting parameters were not found! Residual loss is {}".format(min_loss))
    return w_opt

In [ ]:
import numpy as np
import random as rnd
from copy import deepcopy
from qiskit.quantum_info import Operator
from scipy.optimize import minimize
from src.xyz_evolution import XYZEvolutionCircuit
from qiskit.circuit import ParameterVector

import numpy as np
import random as rnd

from copy import deepcopy
from qiskit.quantum_info import Operator
from scipy.optimize import minimize
from src.xyz_evolution import XYZEvolutionCircuit
from qiskit.circuit import ParameterVector

class CircuitCompressor:
    """
    A class that implements the YBE-powered QTD circuit compression scheme.
    """
    def __init__(self, xyz_evolution_qc):
        self.deep_qc = xyz_evolution_qc

    def get_ybe_update(self, params, l2r=True):
        r"""
        Update $3$-qubit time propagator circuit parameters according to the
        Yang-Baxter Equation (YBE).


        INPUT:

            - ``params`` -- a (3, 2) array describing the parameters for each
              of the three blocks
            - ``l2r`` -- a boolean indicating whether we are applying the symmetry
              left-to-right or vice versa

        OUTPUT:

            A (3, 2) NumPy array describing the parameters of an equivalent circuit. 
        """
        if l2r:
            # Get equivalent parametrized circuit
            equiv_qc = XYZEvolutionCircuit(3)
            params = np.reshape(params,[3,2])
            result_params = [ParameterVector('theta'+str(k),2) for k in range(3)]
            for index,param in enumerate(result_params):
                index = index%(equiv_qc.num_qubits-1)
                if index<int(equiv_qc.num_qubits/2):
                    equiv_qc.uxz(*param, 2*index, 2*index + 1)
                else:
                    index = index - int(equiv_qc.num_qubits/2)
                    equiv_qc.uxz(*param, 2*index+1, 2*index + 2)


            # Construct target unitary operator
            target_qc = XYZEvolutionCircuit(3)
            
            for index,param in enumerate(params):
                index = index%(target_qc.num_qubits-1)
                if index<int(target_qc.num_qubits/2-1):
                    target_qc.uxz(*param, 2*index+1, 2*index + 2)
                else:
                    index = index - int(equiv_qc.num_qubits/2)
                    target_qc.uxz(*param, 2*index, 2*index +1)

            target = Operator(target_qc).data

            # Get parameters for the equivalent time propagator
            new_params = get_optimized_params(equiv_qc, target)
            #new_params = np.reshape(new_params,[3,2])
            return new_params
        else:
            # Get equivalent parametrized circuit
            equiv_qc = XYZEvolutionCircuit(3)
            params = np.reshape(params,[3,2])
            result_params = [ParameterVector('theta'+str(k),2) for k in range(3)]
            for index,param in enumerate(result_params):
                index = index%(equiv_qc.num_qubits-1)
                if index<int(equiv_qc.num_qubits/2):
                    equiv_qc.uxz(*param, 2*index+1, 2*index + 2)
                else:
                    index = index - int(equiv_qc.num_qubits/2)
                    equiv_qc.uxz(*param, 2*index, 2*index + 1)


            # Construct target unitary operator
            target_qc = XYZEvolutionCircuit(3)
            
            for index,param in enumerate(params):
                index = index%(target_qc.num_qubits-1)
                if index<int(target_qc.num_qubits/2):
                    target_qc.uxz(*param, 2*index, 2*index + 1)
                else:
                    index = index - int(equiv_qc.num_qubits/2)
                    target_qc.uxz(*param, 2*index+1, 2*index +2)

            target = Operator(target_qc).data

            # Get parameters for the equivalent time propagator
            new_params = get_optimized_params(equiv_qc, target)
            #new_params = np.reshape(new_params,[3,2])
            return new_params
            

    def get_mirror_update(self, params, l2r=True):
        r"""
        Update the $4$-qubit time propagator circuit parameters following the
        so-called mirror step.

        INPUT:

            - ``params`` -- a (6, 2) NumPy array describing the parameters for
            each of the six blocks
            - ``l2r`` -- a boolean indicating whether we are applying the symmetry
            left-to-right or vice versa
        
        OUTPUT:

            A (6, 2) NumPy array describing the parameters of the mirrored circuit.
        """
        # Number of parameters per block
        if l2r == False:
            equiv_qc = XYZEvolutionCircuit(4)
            equiv_qc._construct_evolution_qc('XZ', num_layers=4,odd=False, bound=False)
            # Construct target unitary operator
            target_qc = XYZEvolutionCircuit(4)
            target_qc._construct_evolution_qc('XZ', num_layers=4,odd=True, bound=False)
            target_qc = target_qc.assign_parameters(params)
            target = Operator(target_qc).data
        else:
            equiv_qc = XYZEvolutionCircuit(4)
            equiv_qc._construct_evolution_qc('XZ', num_layers=4,odd=True, bound=False)
            # Construct target unitary operator
            target_qc = XYZEvolutionCircuit(4)
            target_qc._construct_evolution_qc('XZ', num_layers=4,odd=False, bound=False)
            target_qc = target_qc.assign_parameters(params)
            target = Operator(target_qc).data
            

        # Get parameters for the equivalent time propagator
        new_params = get_optimized_params(equiv_qc, target)
        
        return new_params


    def compress_circuit(self):
        """
        Compress this time evolution circuit using the YBE.

        OUTPUT:

            Returns a compressed :class:`.XYZEvolutionCircuit` object
            with $N (N - 1)/2$ blocks that is equivalent to the uncompressed
            Trotterization circuit ``self.deep_qc``.
        """
        # Return if deep_qc is empty
        qc = self.deep_qc
        if not list(qc):
            return qc

        # Extract parameters
        bsz = 2
        evol_params = qc.time_delta * qc.coupling_const[np.nonzero(qc.coupling_const)]

        N = qc.num_qubits
        w = np.zeros(N*(N-1))
        if(N == 3):
            num_layers = 2 * self.deep_qc.trotter_num
            num_blocks = ceil(num_layers * (self.deep_qc.num_qubits - 1) / 2)
            params = list(evol_params)*num_blocks
            for i in range(num_blocks-int(N*(N-1)/2)):
                if i == 0:
                    update = self.get_ybe_update(params[0:int(N*(N-1))])
                    update[4] += evol_params[0]
                    update[5] += evol_params[1] 
                else:
                    update = self.get_ybe_update(update)
                    update[4] += evol_params[0]
                    update[5] += evol_params[1]
            if (num_blocks-3)%2 == 1:
                compressed = XYZEvolutionCircuit(N)
                compressed._construct_evolution_qc(qc.propagator, num_layers=N,odd=True, bound=False)
            else:
                compressed = XYZEvolutionCircuit(N)
                compressed._construct_evolution_qc(qc.propagator, num_layers=N, bound=False)    
            w = update
            return compressed.assign_parameters(w)
            
        if(N == 4):
            num_layers = 2 * self.deep_qc.trotter_num
            num_blocks = ceil(num_layers * (self.deep_qc.num_qubits - 1) / 2)
            params = list(evol_params)*num_blocks
            for i in range(num_layers-4):
                if i == 0:
                    update = self.get_mirror_update(params[0:int(N*(N-1))])
                    update[8] += evol_params[0]
                    update[9] += evol_params[1] 
                    update[10] += evol_params[0] 
                    update[11] += evol_params[1] 
                elif i%2 == 1:
                    update = self.get_mirror_update(update,False)
                    update[10] += evol_params[0]
                    update[11] += evol_params[1]
                else:
                    update = self.get_mirror_update(update)
                    update[8] += evol_params[0]
                    update[9] += evol_params[1] 
                    update[10] += evol_params[0] 
                    update[11] += evol_params[1] 
                    
            if (num_layers-4)%2 == 1:
                compressed = XYZEvolutionCircuit(N)
                compressed._construct_evolution_qc(qc.propagator, num_layers=N,odd=True, bound=False)
            else:
                compressed = XYZEvolutionCircuit(N)
                compressed._construct_evolution_qc(qc.propagator, num_layers=N, bound=False)    
                
            w = update
        # Construct compressed circuit
            return compressed.assign_parameters(w)

#########################
### Parameter fitting ###
#########################

# Minimize quadratic target loss... Attempt max_shot times using random initial point
def get_optimized_params(param_circ, target_unitary, res_tol=1e-7, max_shots=10, verbose=False):
    # Define quadratic loss for fitting
    loss = lambda p: np.linalg.norm(Operator(param_circ.assign_parameters(p)).data - target_unitary)
    n_params = param_circ.num_parameters
    min_loss = 100
    w_opt = 0
    for k in range(max_shots):
        w0 = [2*np.pi*rnd.random()+np.pi for j in range(n_params)]
        opt = minimize(
                loss, w0, method='L-BFGS-B', bounds=[(0, 4*np.pi)]*n_params,
                jac='3-point', options={'ftol': 1e-10}
            )
        if opt.fun < min_loss:
            min_loss = opt.fun
            w_opt = opt.x
        if min_loss < res_tol:
            return w_opt
    if verbose:
        print("WARNING: Good fitting parameters were not found! Residual loss is {}".format(min_loss))
    return w_opt

In [19]:
from src.xyz_evolution import XYZEvolutionCircuit
import numpy as np
from math import ceil
from src.circuit_compressor import CircuitCompressor
temp = CircuitCompressor(XYZEvolutionCircuit(4,np.array([1,0,1]),trotter_num=4,final_time=2)).compress_circuit()

In [3]:
from qiskit.quantum_info import Operator

In [21]:
temp = CircuitCompressor(XYZEvolutionCircuit(4,np.array([1,0,1]),trotter_num=4,final_time=2)).compress_circuit()

In [22]:
temp.draw()

┌─────────────────────┐                       ┌─────────────────────┐»
q_0: ┤0                    ├───────────────────────┤0                    ├»
     │  Uxz(10.369,4.0857) │┌─────────────────────┐│  Uxz(4.2979,4.2979) │»
q_1: ┤1                    ├┤0                    ├┤1                    ├»
     ├─────────────────────┤│  Uxz(5.1061,7.4603) │├─────────────────────┤»
q_2: ┤0                    ├┤1                    ├┤0                    ├»
     │  Uxz(4.0857,7.2273) │└─────────────────────┘│  Uxz(4.2979,7.4394) │»
q_3: ┤1                    ├───────────────────────┤1                    ├»
     └─────────────────────┘                       └─────────────────────┘»
«                            
«q_0: ───────────────────────
«     ┌─────────────────────┐
«q_1: ┤0                    ├
«     │  Uxz(7.1061,7.1061) │
«q_2: ┤1                    ├
«     └─────────────────────┘
«q_3: ───────────────────────
«

In [20]:
np.linalg.norm(Operator(temp)-Operator(XYZEvolutionCircuit(4,np.array([1,0,1]),trotter_num=4,final_time=2)))

7.781111411323964e-10

In [ ]:
XYZEvolutionCircuit(4,np.array([1,0,1]),trotter_num=4,final_time=2).draw()

┌──────┐        ┌──────┐        ┌──────┐        ┌──────┐        
q_0: ┤0     ├────────┤0     ├────────┤0     ├────────┤0     ├────────
     │  Uxz │┌──────┐│  Uxz │┌──────┐│  Uxz │┌──────┐│  Uxz │┌──────┐
q_1: ┤1     ├┤0     ├┤1     ├┤0     ├┤1     ├┤0     ├┤1     ├┤0     ├
     ├──────┤│  Uxz │├──────┤│  Uxz │├──────┤│  Uxz │├──────┤│  Uxz │
q_2: ┤0     ├┤1     ├┤0     ├┤1     ├┤0     ├┤1     ├┤0     ├┤1     ├
     │  Uxz │└──────┘│  Uxz │└──────┘│  Uxz │└──────┘│  Uxz │└──────┘
q_3: ┤1     ├────────┤1     ├────────┤1     ├────────┤1     ├────────
     └──────┘        └──────┘        └──────┘        └──────┘